In [1]:
### 
# Sheets:
# 1 - Experimental Setup
# 2 - Basic Features
# 3 - PCA, All Together
# 4 - PCA, Individual Reps
# 5 - Histograms
# 6 - Clustering / Heatmaps

In [2]:
import pandas as pd
import numpy as np
from numpy.polynomial.polynomial import polyfit
import math
import sys
import os
import matplotlib.colors as mat_colors
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg 
import re
import experiment as molp5
import scipy
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
plt.rc('text', usetex=True)
plt.rcParams.update({'font.size': 16})

In [4]:
raw_data_rep1 = pd.read_csv("rep1_Image.csv",sep=r'\s*,\s*',engine="python")
raw_data_rep2 = pd.read_csv("rep2_Image.csv",sep=r'\s*,\s*',engine="python")
raw_data_rep3 = pd.read_csv("rep3_Image.csv",sep=r'\s*,\s*',engine="python")
plate_map = pd.read_csv("plate_0000107.csv",sep=r'\s*,\s*',engine="python")
control_map = pd.read_csv("plate_0000108.csv",sep=r'\s*,\s*',engine="python")
control_data_rep1 = pd.read_csv("ctrl_rep1_Image.csv",sep=r'\s*,\s*',engine="python")
control_data_rep2 = pd.read_csv("ctrl_rep2_Image.csv",sep=r'\s*,\s*',engine="python")
control_data_rep3 = pd.read_csv("ctrl_rep3_Image.csv",sep=r'\s*,\s*',engine="python")

In [5]:
# Some information about the experiment
sites = 4
excluded_rows = ['a', 'b', 'o', 'p']
excluded_cols = [1, 2, 23, 24]

In [46]:
def dropColumns(dataframe, *args, **kwargs):
    ax = kwargs.pop('axis', 0)
    for arg in args:
        dataframe = dataframe.copy().drop(arg, ax)
    return dataframe

class Experiment:
    def __init__(self, raw_data, plate_map, sites, excluded_rows, excluded_cols):
        self.raw_data = raw_data
        self.raw_plate = plate_map
        self.sites = sites
        self.excluded_rows = excluded_rows
        self.excluded_cols = excluded_cols
        self.replicates = len(self.raw_data)
        self.processed_raw_data = self.preprocessRawData()
        # We may have N replicates and 1 plate map for all of them, so we don't need to call
        # preprocessWellPlate with the entire list of replicates. We just need it temporarily to
        # get the list of wells, and we use the list of wells to standardize the wells on the
        # plate map, so passing in one is just fine.
        self.processed_plate = self.preprocessWellPlate(
            self.processed_raw_data[0], self.raw_plate, self.sites, self.excluded_rows, self.excluded_cols
        )
        self.dmso_wells = self.getDMSOWells()
        self.compound_wells = self.getCompoundWells()
        self.dmso_replicates = self.getDMSORows()
        self.molecules = list(set(list(self.processed_plate[self.processed_plate['xk_sample_id'].notna()]['xk_sample_id'])))
        self.molecules = [mol for mol in self.molecules if mol != "XK00000397-001"]
        self.concentrs = list(set(list(self.processed_plate[self.processed_plate['concentration'].notna()]['concentration'])))
        self.num_molecules = len(self.molecules)
        self.num_concentrs = len(self.concentrs)
        self.concentrs.sort()
        self.concentrs.reverse()
        self.dmso_reps_mean = self.collapseDMSO('mean')
        self.dmso_reps_stdev = self.collapseDMSO('stdev')
        self.features_daccess = dropColumns(
            self.dmso_replicates[0], 'Metadata_Well', 'Count_Cells', 'Count_Cytoplasm', 'Count_Nuclei', axis=1
        ).columns
        self.features_human = [feature.replace('_', ' ') for feature in self.features_daccess]
        self.dmso_cell_counts = self.getDMSOCellCounts()
        self.__generateFingerprintMatrices()
        self.__generateDistanceMatrices('euclidean')
        self.__getAverageOfReplicates()
        self.__getVarianceOfReplicates()

    def preprocessRawData(self):
        processed_data = []
        for entry in self.raw_data:
            raw_wells = self.condenseWells(entry)
            raw_numeric = self.filterNonnumericData(entry)
            feature_counts = self.getFeatureCounts(entry, self.sites)
            features_and_wells = self.mergeDataframes(feature_counts, raw_wells)
            cols = list(features_and_wells.columns.values)
            cols = [cols[-1]] + cols[:3]
            features_and_wells = features_and_wells[cols]
            aggregated_data = self.getMeanData(raw_numeric)
            # aggregated_median = self.getMedianData(raw_numeric)
            # aggregated_stdev = self.getStdevData(raw_numeric)
            # aggregated_data = self.mergeDataframes(self.mergeDataframes(aggregated_mean, aggregated_median),aggregated_stdev)
            aggregated_data_merged = aggregated_data.groupby(np.arange(len(aggregated_data))//self.sites).mean()
            correlated_data = self.mergeDataframes(features_and_wells, aggregated_data_merged)
            processed_data.append(correlated_data)
        return processed_data

    def preprocessWellPlate(self, processed_raw_wells, plate_map,
                            sites = 4,
                            plate_excluded_rows = ['a', 'b', 'c', 'd'],
                            plate_excluded_cols = [1,2,23,24]):
        plate_filtered = self.filterWellPlate(self.raw_plate)
        plate_well_mapped = dropColumns(
            plate_filtered, 'well_row', 'well_column', 'plate_barcode', 'layout_type', axis=1)
        plate_well_mapped['well'] = processed_raw_wells['Metadata_Well']
        # Rearrange columns so wells come first
        cols = list(plate_well_mapped.columns.values)
        cols = [cols[-1]] + cols[:-1]
        plate_well_mapped = plate_well_mapped[cols]
        return plate_well_mapped

    def condenseWells(self, dataframe):
        return dataframe["Metadata_Well"].iloc[::self.sites].reset_index(drop=True)

    def getFeatureCounts(self, dataframe, sites):
        raw_features = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('Count_'))]].copy()
        return raw_features.groupby(np.arange(len(raw_features))//sites).sum()

    def mergeDataframes(self, becomes_leading, becomes_trailing):
        return becomes_leading.merge(becomes_trailing, how='outer', left_index=True, right_index=True)

    def getMeanData(self, dataframe):
        dataframe = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('Mean'))]].copy()
        dataframe_cell = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('Mean_Cell'))]].copy()
        dataframe_cyto = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('Mean_Cyto'))]].copy()
        dataframe_nucl = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('Mean_Nuclei'))]].copy()
        dataframe_cell_merged = dataframe_cell.groupby(np.arange(len(dataframe_cell))//self.sites).mean()
        dataframe_cyto_merged = dataframe_cyto.groupby(np.arange(len(dataframe_cyto))//self.sites).mean()
        dataframe_nucl_merged = dataframe_nucl.groupby(np.arange(len(dataframe_nucl))//self.sites).mean()
        return self.mergeDataframes(self.mergeDataframes(dataframe_cell, dataframe_cyto), dataframe_nucl)

    def getMedianData(self, dataframe):
        dataframe = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('Median'))]].copy()
        dataframe_cell = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('Median_Cell'))]].copy()
        dataframe_cyto = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('Median_Cyto'))]].copy()
        dataframe_nucl = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('Median_Nuclei'))]].copy()
        dataframe_cell_merged = dataframe_cell.groupby(np.arange(len(dataframe_cell))//self.sites).mean()
        dataframe_cyto_merged = dataframe_cyto.groupby(np.arange(len(dataframe_cyto))//self.sites).mean()
        dataframe_nucl_merged = dataframe_nucl.groupby(np.arange(len(dataframe_nucl))//self.sites).mean()
        return self.mergeDataframes(self.mergeDataframes(dataframe_cell, dataframe_cyto), dataframe_nucl)

    def getStdevData(self, dataframe):
        dataframe = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('StDev'))]].copy()
        dataframe_cell = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('StDev_Cell'))]].copy()
        dataframe_cyto = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('StDev_Cyto'))]].copy()
        dataframe_nucl = dataframe[dataframe.columns[pd.Series(dataframe.columns.str.startswith('StDev_Nuclei'))]].copy()
        dataframe_cell_merged = dataframe_cell.groupby(np.arange(len(dataframe_cell))//self.sites).mean()
        dataframe_cyto_merged = dataframe_cyto.groupby(np.arange(len(dataframe_cyto))//self.sites).mean()
        dataframe_nucl_merged = dataframe_nucl.groupby(np.arange(len(dataframe_nucl))//self.sites).mean()
        return self.mergeDataframes(self.mergeDataframes(dataframe_cell, dataframe_cyto), dataframe_nucl)

    def filterNonnumericData(self, raw_data):
        return raw_data._get_numeric_data()

    def mergeDataframes(self, becomes_leading, becomes_trailing):
        return becomes_leading.merge(becomes_trailing, how='outer', left_index=True, right_index=True)

    def filterWellPlate(self, plate_map):
        filter_rows = plate_map[~plate_map['well_row'].isin(self.excluded_rows)].reset_index(drop=True)
        filter_both = filter_rows[~filter_rows['well_column'].isin(self.excluded_cols)].reset_index(drop=True)
        return filter_both

    def getRaw(self, replicate):
        return self.raw_data[replicate]

    def getProcessed(self, replicate):
        return self.processed_raw_data[replicate]

    def getDMSOWells(self):
        return list(dropColumns(self.processed_plate[(self.processed_plate["solvent"].notna()) & (self.processed_plate["xk_sample_id"].isnull())].copy(),
                                'xk_sample_id', 'concentration', 'concentration_units', axis=1)['well'])

    def getDMSORows(self):
        dmso_reps = []
        for entry in self.processed_raw_data:
            dmso_reps.append(entry[entry['Metadata_Well'].isin(self.dmso_wells)].reset_index(drop=True))
        return dmso_reps

    def getCompoundWells(self):
        return self.processed_plate[(self.processed_plate["xk_sample_id"].notna())].copy().reset_index(drop=True)

    def mergeSitesToWell(self, dataframe, sites=4):
        return dataframe.groupby(np.arange(len(dataframe))//sites).mean()

    def collapseToOneRow(self, dataframe, method):
        if(method == 'variance'):
            return dataframe.copy().groupby(np.arange(len(dataframe))//len(dataframe)).var()
        elif(method == 'mean'):
            return dataframe.copy().groupby(np.arange(len(dataframe))//len(dataframe)).mean()
        elif(method == 'stdev'):
            return dataframe.copy().groupby(np.arange(len(dataframe))//len(dataframe)).std()
        else:
            raise ValueError('Enter a method -- one of: variance, mean, stdev')

    def getDMSOCellCounts(self):
        dmso_avg_with_counts = MyTest.collapseDMSOWithCounts('mean')
        counts = []
        for entry in dmso_avg_with_counts:
            counts.append(entry['Count_Cells'].to_list()[0])
        return counts

    def collapseDMSOWithCounts(self, method):
        reps = []
        for entry in self.dmso_replicates:
            reps.append(self.collapseToOneRow(entry, method))
        return reps

    def collapseDMSO(self, method):
        reps = []
        for entry in self.dmso_replicates:
            reps.append(dropColumns(self.collapseToOneRow(entry, method), 'Count_Cells', 'Count_Cytoplasm', 'Count_Nuclei', axis=1))
        return reps

    def getWellByCompoundConc(self, compound, concentration):
        compound_well_with_conc = self.compound_wells.loc[(self.compound_wells['xk_sample_id'].astype('str')==compound) &
                                                 (self.compound_wells['concentration'] == concentration)]
        return compound_well_with_conc

    def findWellInRawData(self, well, replicate):
        python_wants_a_list = [well]
        return (self.processed_raw_data[replicate])[(self.processed_raw_data[replicate])['Metadata_Well'].isin(python_wants_a_list)].reset_index(drop=True)

    def getCompoundRows(self, replicate):
        return (self.processed_raw_data[replicate])[(self.processed_raw_data[replicate])['Metadata_Well'].isin(self.compound_wells['well'].to_list())].reset_index(drop=True)

    def getObsCounts(self, compound, conc, replicate):
        if(replicate < 1) or (replicate > self.replicates):
            raise ValueError("Replicate must be in range [1,reps]")
        else:
            compound_data = self.getWellByCompoundConc(compound, conc)
            if compound_data.empty:
                return None
            well = compound_data['well'].reset_index(drop=True)[0]
            conc = compound_data['concentration'].reset_index(drop=True)[0]
            compound_data = self.findWellInRawData(well, (replicate - 1))
            cell_count = compound_data['Count_Cells'].reset_index(drop=True)[0]
            cyto_count = compound_data['Count_Cytoplasm'].reset_index(drop=True)[0]
            nucl_count = compound_data['Count_Nuclei'].reset_index(drop=True)[0]
            temp_df = pd.DataFrame()
            temp_df["Replicate"] = [replicate]
            temp_df["Molecule"] = [compound]
            temp_df["Concentration"] = [conc]
            temp_df["Count_Cells"] = [cell_count]
            temp_df["Count_Cells_Scaled"] = [cell_count / self.dmso_cell_counts[(replicate - 1)]]
            temp_df["Count_Cytoplasm"] = [cyto_count]
            temp_df["Count_Nuclei"] = [nucl_count]
            return temp_df

    def getCompoundFingerprint(self, compound, conc, replicate, rtype='None', keepCounts=False):
        if(replicate < 1) or (replicate > self.replicates):
            raise ValueError("Replicate must be in range [1,reps]")
        else:
            compound_data = self.getWellByCompoundConc(compound, conc)
            if compound_data.empty:
                return None
            well = compound_data['well'].reset_index(drop=True)[0]
            conc = compound_data['concentration'].reset_index(drop=True)[0]
            compound_data = self.findWellInRawData(well, (replicate - 1))
            cell_count = compound_data['Count_Cells'].reset_index(drop=True)[0]
            cyto_count = compound_data['Count_Cytoplasm'].reset_index(drop=True)[0]
            nucl_count = compound_data['Count_Nuclei'].reset_index(drop=True)[0]
            new_data = dropColumns(compound_data, 'Metadata_Well', 'Count_Cells', 'Count_Cytoplasm', 'Count_Nuclei', axis=1)
            dmso_data_mean = self.dmso_reps_mean[(replicate - 1)]
            dmso_data_stdv = self.dmso_reps_stdev[(replicate - 1)]
            if(rtype == 'list'):
                return list(((new_data - dmso_data_mean) / dmso_data_stdv).loc[0])
            else:
                if(keepCounts == True):
                    temp_df = (new_data - dmso_data_mean) / dmso_data_stdv
                    temp_df["Concentration"] = conc
                    temp_df["Cell_Count"] = cell_count
                    return temp_df
                else:
                    return (new_data - dmso_data_mean) / dmso_data_stdv

    def getSingleDistance(self, compound, concentration, replicate):
        if concentration not in self.concentrs:
            raise ValueError("Concentration not found in experiment.")
        else:
            this_fp = self.getCompoundFingerprint(compound, concentration, replicate)
            if this_fp is None:
                return None
            else:
                return [concentration, math.sqrt(self.getCompoundFingerprint(compound, concentration, replicate).pow(2).sum(axis=1))]

    def getAllDistances(self, compound, replicate):
        this_vector = []
        for conc in self.concentrs:
            try:
                this_vector.append(self.getSingleDistance(compound, conc, replicate))
            # If this molecule was not tested at a particular concentration
            except:
                pass
        return this_vector

    def generateVectorDistances(self, replicate=None):
        distances = pd.DataFrame()
        if replicate is None:
            for rep in range(1, self.replicates+1):
                rep_df = pd.DataFrame()
                for molecule in self.molecules:
                    for conc in self.concentrs:
                        temp_df = pd.DataFrame()
                        temp_df["Replicate"] = [rep]
                        temp_df["Molecule"] = [molecule]
                        this_dist = self.getSingleDistance(molecule, conc, rep)
                        if this_dist is None:
                            continue
                        else:
                            temp_df["Concentration"] = [this_dist[0]]
                            temp_df["Magnitude"] = [this_dist[1]]
                            rep_df = rep_df.append(temp_df)
                distances = distances.append(rep_df)
            return distances
        else: 
            rep_df = pd.DataFrame()
            for molecule in self.molecules:
                for conc in self.concentrs:
                    temp_df = pd.DataFrame()
                    temp_df["Replicate"] = [replicate]
                    temp_df["Molecule"] = [molecule]
                    this_dist = self.getSingleDistance(molecule, conc, replicate)
                    if this_dist is None:
                        continue
                    else:
                        temp_df["Concentration"] = [this_dist[0]]
                        temp_df["Magnitude"] = [this_dist[1]]
                        rep_df = rep_df.append(temp_df)
            distances = distances.append(rep_df)
            return distances


    def getExtremesInFingerprintSeries(self, compound, conc, replicate):
        max_y = -sys.maxsize - 1
        min_y = sys.maxsize
        for conc in self.concentrs:
            this_replicate = self.getCompoundFingerprint(compound, conc, replicate, 'list')
            if this_replicate is not None:
                this_min = min(this_replicate)
                this_max = max(this_replicate)
                if(this_min < min_y): 
                    min_y = this_min
                if(this_max > max_y):
                    max_y = this_max
        return [math.floor(min_y), math.ceil(max_y)]

    def getExtremesAcrossReplicates(self, compound, conc):
        max_y = -sys.maxsize - 1
        min_y = sys.maxsize
        for replicate in range(1,self.replicates+1):
            this_replicate = self.getCompoundFingerprint(compound, conc, replicate, 'list')
            if this_replicate is not None:
                this_min = min(this_replicate)
                this_max = max(this_replicate)
                if(this_min < min_y): 
                    min_y = this_min
                if(this_max > max_y):
                    max_y = this_max
        return [math.floor(min_y), math.ceil(max_y)]

    def __generateFingerprintMatrices(self):
        self.replicate_fingerprint_frames = []
        for rep in range(1, self.replicates + 1):
            fingerprints = pd.DataFrame()
            for molecule in self.molecules:
                for conc in self.concentrs:
                    this_fp = pd.DataFrame()
                    this_fp["Replicate"] = [rep]
                    this_fp["Molecule"] = [molecule]
                    this_fp['Concentration'] = [conc]
                    this_data = self.getCompoundFingerprint(molecule, conc, rep)
                    if this_data is None:
                        continue
                    else:
                        final = this_fp.merge(this_data, how='outer', left_index=True, right_index=True)
                        fingerprints = fingerprints.append(final)
            self.replicate_fingerprint_frames.append(fingerprints)

    def getFingerprintMatrix(self, replicate, aio=False):
        if self.replicate_fingerprint_frames is None:
            self.__generateFingerprintMatrices()
            return self.replicate_fingerprint_frames[replicate - 1].reset_index(drop=True)
        else:
            if(aio == False):
                return self.replicate_fingerprint_frames[replicate - 1].reset_index(drop=True)
            else:
                temp = pd.DataFrame()
                for rep in replicate_fingerprint_frames:
                    temp = temp.append(rep)
                return temp.reset_index(drop=True)

    def __generateDistanceMatrices(self, method):
        self.replicate_euclidean_distance_matrices = []
        for rep in self.replicate_fingerprint_frames:
            temp = dropColumns(rep, 'Replicate', 'Molecule', 'Concentration', axis=1)
            if(method == 'euclidean'):
                self.replicate_euclidean_distance_matrices.append(scipy.spatial.distance_matrix(temp, temp))
            if(method == 'mahalanobis'):
                self.replicate_mahalanobis_distance_matrices.append(scipy.spatial.distance_matrix(temp, temp))

    def getDistanceMatrix(self, replicate, method='euclidean'):
        if self.replicate_fingerprint_frames is None:
            self.__generateFingerprintMatrices()
        if(method == 'euclidean'):
            if self.replicate_euclidean_distance_matrices is None:
                self.__generateDistanceMatrices('euclidean')
            return self.replicate_euclidean_distance_matrices[replicate - 1]
        elif(method == 'mahalanobis'):
            if self.replicate_mahalanobis_distance_matrices is None:
                self.__generateDistanceMatrices('mahalanobis')
            return self.replicate_mahalanobis_distance_matrices[replicate - 1]
        else:
            raise ValueError("Must specify method as one of: euclidean, mahalanobis")

    def __getAverageOfReplicates(self):
        molecules = pd.DataFrame(data=self.replicate_fingerprint_frames[0]['Molecule']).reset_index(drop=True)
        temp_replicates = [dropColumns(rep, 'Replicate', 'Molecule', axis=1) for rep in self.replicate_fingerprint_frames]
        temp_df = (sum(temp_replicates) / 3).reset_index(drop=True)
        self.replicate_fingerprint_frames_averaged = self.mergeDataframes(molecules, temp_df)

    def __getVarianceOfReplicates(self):
        temp_df = self.getFingerprintMatrix(1)
        molecules = temp_df['Molecule']
        conc = temp_df['Concentration']
        for i in range(2, self.replicates+1):
            temp_df = temp_df.append(self.getFingerprintMatrix(i))
        final_df = temp_df.copy().groupby([i % len(self.getFingerprintMatrix(1)) for i in range(len(temp_df))]).var()
        final_df['Molecule'] = molecules
        final_df['Concentration'] = conc
        final_df = dropColumns(final_df, 'Replicate', axis=1)
        cols = list(final_df.columns.values)
        cols = [cols[-1]] + cols[:-1]
        final_df = final_df[cols]
        self.replicate_fingerprint_frames_variance = final_df

    def getAverageOfReplicates(self):
        return self.replicate_fingerprint_frames_averaged

    def getVarianceOfReplicates(self):
        return self.replicate_fingerprint_frames_variance

    sites = None
    replicates = None
    excluded_rows = None
    excluded_cols = None
    molecules = None
    concentrs = None
    features_human = None
    features_daccess = None
    num_molecules = None
    num_concentrs = None

    raw_data = None
    raw_plate = None

    replicate_fingerprint_frames = None
    replicate_euclidean_distance_matrices = None
    replicate_mahalanobis_distance_matrices = None

    replicate_fingerprint_frames_averaged = None
    replicate_fingerprint_frames_variance = None
    replicate_cov_matrix = None

    replicate_similarity_matrices = None

    processed_raw_data = None
    processed_plate = None

    dmso_wells = None
    compound_wells = None

    dmso_cell_counts = None
    dmso_replicates = None
    dmso_reps_mean = None
    dmso_reps_stdev = None

class DistanceMatrix:
    def __init__(self, distances):
        self.distances = distances

    distances = None

class Molecule:
    def __init__(self, name, replicates, distances):
        self.name = name
        self.replicates = replicates
        self.distances = distances

    def getDistances(self, replicate):
        if(replicate < 1):
            raise ValueError("Replicate 0?")
        elif(replicate > self.replicates):
            raise ValueError("Index out of bounds.")
        else:
            return distances[replicate - 1]

    name = None
    replicates = None
    distances = None

In [40]:
MyTest.dmso_cell_counts

[1191.0188679245282, 1174.5849056603774, 2253.3207547169814]

In [47]:
MyTest = Experiment([raw_data_rep1, raw_data_rep2, raw_data_rep3], plate_map, sites, excluded_rows, excluded_cols)
control_exp = Experiment([control_data_rep1, control_data_rep2, control_data_rep3], control_map, sites, excluded_rows, excluded_cols)

In [9]:
rep_df = pd.DataFrame()
for molecule in MyTest.molecules:
    for conc in MyTest.concentrs:
        test = MyTest.getObsCounts(molecule, conc, 1)
        rep_df = rep_df.append(test)

In [48]:
def get_obs_counts(molp5_exp):
    reps = []
    for rep in range(1, molp5_exp.replicates+1):
        rep_df = pd.DataFrame()
        for molecule in molp5_exp.molecules:
            for conc in molp5_exp.concentrs:
                test = molp5_exp.getObsCounts(molecule, conc, rep)
                rep_df = rep_df.append(test)
        reps.append(rep_df)
    return reps

In [ ]:
test3 = get_obs_counts(MyTest)

In [ ]:
rep_df.head

In [13]:
temp_test = rep_df.loc[rep_df['Molecule'] == 'XK00001384-001']
temp_test_con = temp_test['Concentration'].to_list()
temp_test_cell = temp_test['Count_Cells'].to_list()

In [14]:
print(temp_test_con)
print(temp_test_cell)

[50.0, 16.67, 5.56, 1.85, 0.617, 0.20600000000000002]
[766, 1265, 1553, 1561, 1491, 1534]


In [53]:
MyTest.dmso_cell_counts

[1191.0188679245282, 1174.5849056603774, 2253.3207547169814]

In [51]:
exp_reps = get_obs_counts(MyTest)
ctrl_reps = get_obs_counts(control_exp)

In [ ]:
exp_reps

In [ ]:

fig, ax = plt.subplots(figsize=(20,40))
fig.patch.set_facecolor('white')
for molecule in control_exp.molecules:
    for index, entry in enumerate(exp_reps):
        this_mol = entry.loc[entry['Molecule'] == molecule]
        this_mol_con = this_mol['Concentration'].to_list()
        con_processed = [math.log(2*i) for i in this_mol_con]
        this_mol_cell = this_mol['Count_Cells'].to_list()
        ax.scatter(this_mol_con, this_mol_cell, label=molecule)
        plt.annotate(molecule+", Rep " + str(index), (this_mol_con[0],this_mol_cell[0]), fontsize=10)
plt.title("Experimental Cell Count vs. Concentration")
plt.xlabel("Concentration (mM)")
plt.ylabel("Cell Count")
#plt.legend()
plt.savefig("CountConcControl.png")
plt.show()

In [16]:
import svgutils.compose as sc
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
size = fig.get_size_inches().tolist()
size

In [89]:
size.

SyntaxError: invalid syntax (<ipython-input-89-0e26066af742>, line 1)

In [93]:
def convertToFigure(figsize):
    figsize_cm = [i * 2.54 for i in figsize]
    sc_x = str(figsize_cm[0]) + "cm"
    sc_y = str(figsize_cm[1]) + "cm"
    return sc_x, sc_y


In [100]:
convertToFigure([10, 10])

('25.4cm', '25.4cm')

In [229]:
def generateCountConcGraphs(molp5_exp, path_prefix, excluded_reps, scale=False, gridlines=None):
    exp_reps = get_obs_counts(molp5_exp)
    for molecule in molp5_exp.molecules:
        fig, ax = plt.subplots(figsize=(10,10))
        for index, entry in enumerate(exp_reps):
            if (index + 1) not in excluded_reps:
                this_mol = entry.loc[entry['Molecule'] == molecule]
                this_mol_con = this_mol['Concentration'].to_list()
                con_processed = [math.log(2*i, 10) for i in this_mol_con]
                if scale == True:
                    this_mol_cell = this_mol['Count_Cells_Scaled'].to_list()
                else:
                    this_mol_cell = this_mol['Count_Cells'].to_list()
                ax.scatter(con_processed, this_mol_cell, label="Replicate " + str(index+1))
        if scale == True:
            ax.set_ylim(0, 1.75)
        if gridlines == None:
            withgrid = None
        elif gridlines == 'gow':
            withgrid = 'gow'
            plt.grid(b=True, which='major', color='#A1A1A1', linestyle='-')
        elif gridlines == 'wog':
            withgrid = 'wog'
            ax.set_facecolor('#E5E5E5')
            plt.grid(b=True, which='major', color='#FFFFFF', linestyle='-')
        else:
            withgrid = None
        if scale == True:
            plt.title("Toxicity of " + molecule)
            plt.ylabel('Relative Viability (\%)')
        else:
            if path_prefix == 'Ctrl':
                plt.title("Experimental Cell Count vs. Concentration -- " + control_moldict[molecule])
            else:
                plt.title("Experimental Cell Count vs. Concentration -- " + molecule)
            plt.ylabel("Cell Count")
        plt.xlabel("Concentration (log$_{10}$[\\textmu M])")
        plt.legend(
        loc='upper center', bbox_to_anchor=(0.5, -0.1),
            fancybox=True, shadow=True, ncol=5
        )
        if scale == True:
            gtype = 'Scaled'
        else:
            gtype = 'Raw'
        path = 'CountConc/' + path_prefix + '/' + gtype + '/'
        figname = molecule + 'CountConc' + withgrid + '.svg'
        total_path = path + figname
        os.makedirs(os.path.dirname(total_path), exist_ok=True)
        plt.savefig(total_path)
        svg_x, svg_y = convertToFigure(fig.get_size_inches().tolist())
        sc.Figure("20cm", "20cm",
            sc.Panel(
                sc.SVG(total_path).scale(1),
                sc.SVG('Images/' + path_prefix + '/' + molecule + '.svg').scale(1.75).move(100,525)
            )
                 ).save(path + "test" + molecule + withgrid + ".svg")
        plt.close('all')


In [233]:
control_moldict = {
    "XK00000498": "Taxol",
    "XK00001539": "Vincristine",
    "XK00001540": "Latrunculin B",
    "XK00001541": "Etoposide",
    "XK00001542": "Berberine chloride",
    "XK00001543": "Cytochalasin D",
    "XK00001544": "Fenbendazole",
    "XK00001545": "Rotenone",
    "XK00001546": "5-Nitro-2-(3-phenylpropylamino)benzoic acid",
    "XK00001547": "SB 203580",
    "XK00001548": "Methyl Ester of CA-074",
    "XK00001549": "Tetrandrine",
    "XK00001550": "Rapamycin"
}

In [ ]:
generateCountConcGraphs(MyTest, 'Exp', excluded_reps = [3], scale=False, gridlines='wog')
generateCountConcGraphs(MyTest, 'Exp', excluded_reps = [3], scale=True, gridlines='wog')

In [ ]:
generateCountConcGraphs(control_exp, 'Ctrl', excluded_reps = [], scale=False, gridlines='wog')
generateCountConcGraphs(control_exp, 'Ctrl', excluded_reps = [ ], scale=True, gridlines='wog')

In [ ]:
test

In [8]:
def plot_distance_raw(ax, vec, label):
    ax.plot([row[0] for row in vec], [row[1] for row in vec], label=label, marker='o')
    ax.legend()
    return ax

In [9]:
def plot_distance_logx(ax, vec, label):
    ax.plot([math.log(row[0],10) for row in vec], [row[1] for row in vec], label=label, marker='o')
    ax.legend()
    return ax

In [10]:
def plot_distance_logy(ax, vec, label):
    ax.plot([row[0] for row in vec], [math.log(row[1],10) for row in vec], label=label, marker='o')
    ax.legend()
    return ax

In [11]:
def plot_distance_logxlogy(ax, vec, label):
    ax.plot([math.log(row[0],10) for row in vec], [math.log(row[1],10) for row in vec], label=label, marker='o')
    ax.legend()
    return ax

In [ ]:
def plot_fingerprint(ax, fp, label, ymin, ymax):
    ax.plot(range(0, len(fp)), fp, linewidth = 0.8, label=label)
    ax.set_xlim([0, len(fp)])
    ax.set_ylim([ymin,ymax])
    return ax

In [13]:
def plot_doser(ax, xvals, yvals, label):
    #xvals = [math.log(val,10) for val in xvals]
    ax.plot(xvals, yvals, label=label, marker='o')
    ax.set_xlim([min(xvals),max(xvals)])
    ax.set_ylim([min(yvals),max(yvals)])
    return ax

In [14]:
def GenerateFingerprintCSVs(molp5_exp, path_prefix, consolidate=False):
    if consolidate:
        fingerprints = pd.DataFrame()
    for rep in range(1, molp5_exp.replicates + 1):
        if not consolidate:
            fingerprints = pd.DataFrame()
        for molecule in molp5_exp.molecules:
            for conc in molp5_exp.concentrs:
                this_fp = pd.DataFrame()
                if consolidate:
                    this_fp["Replicate"] = [rep]
                this_fp["Molecule"] = [molecule]
                this_fp['Concentration'] = [conc]
                this_data = molp5_exp.getCompoundFingerprint(molecule, conc, rep)
                if this_data is None:
                    continue
                else:
                    final = this_fp.merge(this_data, how='outer', left_index=True, right_index=True)
                    fingerprints = fingerprints.append(final)
        if not consolidate:
            path = path_prefix + str(rep) + '/Fingerprints/' + path_prefix + str(rep) + '.csv'
            os.makedirs(os.path.dirname(path), exist_ok=True)
            fingerprints.to_csv(path, header=True) #, dpi=300)
    if consolidate:
        path = 'Aggregate/' + path_prefix + '_fingerprints.csv'
        os.makedirs(os.path.dirname(path), exist_ok=True)
        fingerprints.to_csv(path, header=True) #, dpi=300)



In [28]:
GenerateFingerprintCSVs(MyTest, 'Reps', consolidate=True)

Saving to aggregate folder


In [29]:
GenerateFingerprintCSVs(control_exp, 'Ctrl', consolidate=True)

Saving to aggregate folder


In [237]:
# Plot all fingerprints, all concentrations overlaid (by-replicate)
def GenerateAllConcFingerprintGraphs(molp5_exp, path_prefix):
    for rep in range(1, molp5_exp.replicates + 1):
        for molecule in molp5_exp.molecules:
            fig, ax = plt.subplots(figsize=(30,10))
            fig.patch.set_facecolor('white')
            if path_prefix == 'Ctrl':
                    plt.title('Fingerprint Overlay of ' + control_moldict[molecule] + '; Replicate' + str(rep))
            else:
                    plt.title('Fingerprint Overlay of ' + molecule + '; Replicate' + str(rep))
            for conc in molp5_exp.concentrs:
                extremes = molp5_exp.getExtremesInFingerprintSeries(molecule, conc, rep)
                this_fp = molp5_exp.getCompoundFingerprint(molecule, conc, rep, 'list')
                if this_fp is None:
                    continue
                else:
                    plot_fingerprint(ax,
                                     molp5_exp.getCompoundFingerprint(molecule, conc, rep, 'list'),
                                     label=f"{conc:.3f}", ymin=extremes[0], ymax=extremes[1])
                    plt.legend(loc='upper right')
                    plt.ylabel('Difference from DMSO')
                    plt.xlabel('Mean Feature')
            path_conc = f"{conc:.2f}".replace('.', ',')
            path = path_prefix + str(rep) + '/Fingerprints/Aggregated/' + molecule + '.png'
            os.makedirs(os.path.dirname(path), exist_ok=True)
            plt.savefig(path, dpi=300)
            plt.close('all')

In [238]:
GenerateAllConcFingerprintGraphs(MyTest, 'Rep')
GenerateAllConcFingerprintGraphs(control_exp, 'Ctrl')

In [241]:
# Plot all fingerprints, one by one
def GenerateSingleConcFingerprints(molp5_exp, path_prefix):
    for rep in range(1, 4):
        for molecule in molp5_exp.molecules:
            for conc in molp5_exp.concentrs:
                fig, ax = plt.subplots(figsize=(10,5))
                fig.patch.set_facecolor('white')
                if path_prefix == 'Ctrl':
                    plt.title('Fingerprint of ' + control_moldict[molecule] + '; Replicate' + str(rep))
                else:
                    plt.title('Fingerprint of ' + molecule + '; Replicate' + str(rep))
                this_fp = molp5_exp.getCompoundFingerprint(molecule, conc, rep, 'list')
                if this_fp is None: continue
                else:
                    this_min = min(this_fp)
                    this_max = max(this_fp)
                    plot_fingerprint(ax,
                                     molp5_exp.getCompoundFingerprint(molecule, conc, rep, 'list'),
                                     label=molecule, ymin=this_min, ymax=this_max)
                    plt.legend()
                    plt.ylabel('Difference from DMSO Average at Feature')
                    plt.xlabel('Feature')
                    path_conc = f"{conc:.2f}".replace('.', ',')
                    path = path_prefix + str(rep) + '/Fingerprints/Single/' + path_conc + '/' + molecule + '.png'
                    os.makedirs(os.path.dirname(path), exist_ok=True)
                    plt.savefig(path, dpi=300)
                    plt.close('all')
#plt.show()

In [ ]:
GenerateSingleConcFingerprints(control_exp, 'Ctrl')

In [ ]:
# For each replicate, plot for each molecule 
for rep in range(1, 4):
    for molecule in MyTest.molecules:
        fig, ax = plt.subplots(figsize=(30,10))
        fig.patch.set_facecolor('white')
        plt.title('Fingerprint Overlay of ' + molecule + '; Replicate' + str(rep))
        for conc in MyTest.concentrs:
            this_fp = MyTest.getCompoundFingerprint(molecule, conc, rep, 'list')
            if this_fp is None: continue
            else:
                plot_fingerprint(ax,
                                 this_fp,
                                 label=str(conc))
                plt.legend()
        path_conc = f"{conc:.2f}".replace('.', ',')
        path = 'Rep' + str(rep) + '/Fingerprints/Aggregated/By-Conc/' + molecule + '.png'
        os.makedirs(os.path.dirname(path), exist_ok=True)
        plt.savefig(path, dpi=300)
        plt.close('all')

In [ ]:
# For each molecule-concentration pair, plot the result for each replicate
# TODO (Urgently): Calculate correlation coefficients for these replicates
# and generate a matrix of them 
def GenerateFPReplicateOverlays(molp5_exp, path_prefix):
    for molecule in molp5_exp.molecules[:3]:
        for conc in molp5_exp.concentrs:
            fig, ax = plt.subplots(figsize=(30,10))
            fig.patch.set_facecolor('white')
            if path_prefix == 'Ctrl':
                plt.title('Fingerprint Overlay of ' + control_moldict[molecule] + ' at ' + f"{conc:.3f}mM")
            else:
                plt.title('Fingerprint Overlay of ' + molecule + ' at ' + f"{conc:.3f}mM")
            for rep in range(1, 4):
                this_fp = molp5_exp.getCompoundFingerprint(molecule, conc, rep, 'list')
                extremes = molp5_exp.getExtremesAcrossReplicates(molecule, conc)
                if this_fp is None:
                    continue
                else:
                    plot_fingerprint(ax,
                                     this_fp,
                                     label="Replicate " + str(rep),
                                     ymin=extremes[0], ymax=extremes[1])
            plt.legend(loc='upper right')
            plt.ylabel('Difference from DMSO Average at Feature')
            plt.xlabel('Feature')
            path_conc = f"{conc:.2f}".replace('.', ',')
            path = 'Aggregate/' + path_prefix + '/Fingerprints/By-Conc/' + path_conc + '/' + molecule + '.png'
            os.makedirs(os.path.dirname(path), exist_ok=True)
            plt.savefig(path, dpi=300)
            plt.close('all')

In [240]:
GenerateFPReplicateOverlays(MyTest, 'Rep')
GenerateFPReplicateOverlays(control_exp, 'Ctrl')


No handles with labels found to put in legend.


No handles with labels found to put in legend.


No handles with labels found to put in legend.


No handles with labels found to put in legend.


No handles with labels found to put in legend.


...

No handles with labels found to put in legend.


No handles with labels found to put in legend.


No handles with labels found to put in legend.


No handles with labels found to put in legend.


No handles with labels found to put in legend.


In [82]:
# log scale on neither axis
for molecule in MyTest.molecules:
    fig, ax = plt.subplots(figsize=(20,10))
    fig.patch.set_facecolor('white')
    plt.title('Size of ' + molecule + ' Vector at Each Concentration')
    for rep in range(1,4):
        plot_distance_raw(ax,
                          MyTest.getAllDistances(molecule, rep),
                          label='Replicate ' + str(rep))
    plt.ylabel('Magnitude')
    plt.xlabel('Concentration')
    plt.legend()
    path = 'Aggregate/Reps/Distances_Raw/' + molecule + '.png'
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, dpi=300)
    plt.close('all')

In [83]:
# log scale on x-axis 
for molecule in MyTest.molecules:
    fig, ax = plt.subplots(figsize=(20,10))
    fig.patch.set_facecolor('white')
    plt.title('Size of ' + molecule + ' Vector at Each Concentration')
    for rep in range(1,4):
        plot_distance_logx(ax,
                      MyTest.getAllDistances(molecule, rep),
                      label='Replicate ' + str(rep))
    plt.ylabel('Magnitude')
    plt.xlabel('Log_{10}(Concentration)')
    plt.legend()
    path = 'Aggregate/Reps/Distances_Logx/' + molecule + '.png'
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, dpi=300)
    plt.close('all')
#plt.show()

In [85]:
# log scale on y-axis
for molecule in MyTest.molecules:
    fig, ax = plt.subplots(figsize=(20,10))
    fig.patch.set_facecolor('white')
    plt.title('Size of ' + molecule + ' Vector at Each Concentration')
    for rep in range(1,4):
        plot_distance_logy(ax,
                      MyTest.getAllDistances(molecule, rep),
                      label='Replicate ' + str(rep))
    plt.ylabel('Log_{10}(Magnitude)')
    plt.xlabel('Concentration')
    plt.legend()
    path = 'Aggregate/Reps/Distances_Logy/' + molecule + '.png'
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, dpi=300)
    plt.close('all')
#plt.show()

In [104]:
# log scale on y-axis
for molecule in MyTest.molecules:
    fig, ax = plt.subplots(figsize=(20,10))
    fig.patch.set_facecolor('white')
    plt.title('Size of ' + molecule + ' Vector at Each Concentration')
    for rep in range(1,4):
        plot_distance_logxlogy(ax,
                      MyTest.getAllDistances(molecule, rep),
                      label='Replicate ' + str(rep))
    plt.ylabel('Log_{10}(Magnitude)')
    plt.xlabel('Log_{10}(Concentration)')
    plt.legend()
    path = 'Aggregate/Reps/Distances_Logx_Logy/' + molecule + '.png'
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, dpi=300)
    plt.close('all')
#plt.show()

In [17]:
# Plot all for comparison
for molecule in MyTest.molecules:
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(40,20))
    # Add big axis for common labels
    fig.patch.set_facecolor('white')
    plt.suptitle('Size of ' + molecule + ' Vector at Each Concentration', fontsize=22)
    for rep in range(1,MyTest.replicates + 1):
        plot_distance_raw(ax1,
                          MyTest.getAllDistances(molecule, rep),
                          label='Replicate ' + str(rep))
        plot_distance_logx(ax2,
                           MyTest.getAllDistances(molecule, rep),
                           label='Replicate ' + str(rep))
        plot_distance_logy(ax3,
                           MyTest.getAllDistances(molecule, rep),
                           label='Replicate ' + str(rep))
        plot_distance_logxlogy(ax4,
                               MyTest.getAllDistances(molecule, rep),
                               label='Replicate ' + str(rep))

    ax1.set_xlabel('Concentration', fontsize=16)
    ax1.set_ylabel('Magnitude', fontsize=16)

    ax2.set_xlabel('Log$_{10}$(Concentration)', fontsize=16)
    ax2.set_ylabel('Magnitude', fontsize=16)

    ax3.set_xlabel('Concentration', fontsize=16)
    ax3.set_ylabel('Log$_{10}$(Magnitude)', fontsize=16)

    ax4.set_xlabel('Log$_{10}$(Concentration)', fontsize=16)
    ax4.set_ylabel('Log$_{10}$(Magnitude)', fontsize=16)

    plt.legend()
    path = 'Aggregate/Reps/4x4DistanceGrid/' + molecule + '.png'
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, dpi=300)
    plt.close()
#plt.show()

In [ ]:
# Dose-response curves
# WARNING: THIS TAKES A TON OF RAM AND A VERY LONG TIME
# I ADVISE YOU DO ONE REPLICATE AT A TIME 
#for rep in range(1, 4):
for molecule in MyTest.molecules:
    print("Generating images for " + molecule + " in replicate " + str(1))
    temp_df = pd.DataFrame()
    concentrations = []
    for conc in MyTest.concentrs:
        temp_df_2 = MyTest.getCompoundFingerprint(molecule, conc, 1)
        if temp_df_2 is None:
            continue
        else:
            temp_df = temp_df.append(temp_df_2)
            concentrations.append(conc)
    concentrations.reverse()
    for index, feature in enumerate(MyTest.features_daccess):
        temp_list = temp_df[feature].to_list()
        fig, ax = plt.subplots(figsize=(10,5))
        fig.patch.set_facecolor('white')
        plt.title('Dose-Response Curve: ' + MyTest.features_human[index] + ' for ' + molecule + ' in Replicate ' + str(1) + ' (Unscaled)')
        plot_doser(ax, concentrations, temp_list, 'test')
        plt.ylabel('Raw Response')
        plt.xlabel('Dose')
        path = 'Rep' + str(1) + '/DoseRCs/By-Molecule/' + molecule + '/' + MyTest.features_daccess[index] + '.png'
        os.makedirs(os.path.dirname(path), exist_ok=True)
        plt.savefig(path) #,dpi=300)
        plt.close('all')

In [21]:
# A dose-response fingerprint is generated like this:
# Take a molecule
# Generate all of its fingerprints at every concentration
# For each feature in the fingerprint
# Take the linear regression of the feature values vs concentration
# Find the M and R^2 values from the linear regression
# Plot M. This shows us which features are positively or negatively associated with concentration.
# TODO (Urgently): Change linear regression to logistic regression
def GenerateDoseResponseFingerprint(molp5_exp, positive_threshold, negative_threshold, r2_threshold, path_prefix):
    for rep in range(1, molp5_exp.replicates+1):
        for molecule in molp5_exp.molecules:
            temp_df = pd.DataFrame()
            m_values = []
            r2_values = []
            concentrations = []
            good_r2_values = []
            very_positive_slopes = []
            very_negative_slopes = []
            fig, ax = plt.subplots(figsize=(20,10))
            fig.patch.set_facecolor('white')
            plt.title('Dose-Response Fingerprint for ' + molecule + ' in Replicate ' + str(rep) + ' (Unscaled)')
            for conc in molp5_exp.concentrs:
                temp_df_2 = molp5_exp.getCompoundFingerprint(molecule, conc, rep)
                if temp_df_2 is None:
                    continue
                else:
                    temp_df = temp_df.append(temp_df_2)
                    concentrations.append(conc)
            concentrations.reverse()
            for index, feature in enumerate(molp5_exp.features_daccess):
                temp_list = temp_df[feature].to_list()
                regression = scipy.stats.linregress(concentrations, temp_list)
                m_values.append(regression.slope)
                r2_values.append(regression.rvalue**2)
                if(regression.slope >= positive_threshold):
                    very_positive_slopes.append([molp5_exp.features_human[index],regression.slope])
                if(regression.slope <= negative_threshold):
                    very_negative_slopes.append([molp5_exp.features_human[index],regression.slope])
                if(regression.slope >= r2_threshold):
                    good_r2_values.append([molp5_exp.features_human[index],regression.rvalue**2])
            ax.scatter(np.arange(0,len(r2_values)), r2_values, label='R$^{2}$')
            ax.scatter(np.arange(0,len(r2_values)), m_values, label='M')
            ax.legend()
            plt.ylabel('M-Value for Response Regression')
            plt.xlabel('Feature')
            path = path_prefix + str(rep) + '/DoseRCFPs/' + molecule + '/' + 'figure.png'
            neg_slopespath = path_prefix + str(rep) + '/DoseRCFPs/' + molecule + '/neg_slopes.txt'
            pos_slopespath = path_prefix + str(rep) + '/DoseRCFPs/' + molecule + '/pos_slopes.txt'
            r2_path = path_prefix + str(rep) + '/DoseRCFPs/' + molecule + '/good_rsqared.txt'
            os.makedirs(os.path.dirname(path), exist_ok=True)
            with open(neg_slopespath, 'w+') as filehandle:
                filehandle.writelines("%s\n" % slope for slope in very_negative_slopes)
                filehandle.close()
            with open(pos_slopespath, 'w+') as filehandle:
                filehandle.writelines("%s\n" % slope for slope in very_positive_slopes)
                filehandle.close()
            with open(r2_path, 'w+') as filehandle:
                filehandle.writelines("%s\n" % r2 for r2 in good_r2_values)
                filehandle.close()
            plt.savefig(path, dpi=300)
            plt.close('all')

In [757]:
GenerateDoseResponseFingerprint(MyTest, 0.7, 0.7, 0.7, 'Ctrl')

In [20]:
GenerateDoseResponseFingerprint(control_exp, 0.7, 0.7, 0.7, 'Ctrl')

In [ ]:
# Returns 187 x 1969 list of all wells that had a molecule in them. Filter by concentration, then cluster
# then filter out a molecule, then cluster
# Then try replicates?
dropColumns(MyTest.getCompoundRows(1), 'Metadata_Well', 'Count_Cells', 'Count_Cytoplasm', 'Count_Nuclei', axis=1)

In [68]:
# Pearson's Correlation Matrix


In [67]:
# Spearman's Correlation Matrix


In [66]:
# Clustering by Pearson

In [65]:
# Clustering by Spearman

In [ ]:
# PCA, All Together

In [ ]:
def GeneratePCAs(molp5_exp):
    fingerprint_matrices = pd.DataFrame()
    features = None
    # Dataset
    X = None
    # Standard Scaler output
    x = None
    # Molecule names
    y = None
    # Concentrations
    z = None
    pca = None
    principalComponents = None
    processed_dataframes = None
    final_dataframes = None
    for i in range(1, molp5_exp.replicates + 1):
        fingerprint_matrices = fingerprint_matrices.append(molp5_exp.getFingerprintMatrix(i)).reset_index(drop=True)
    features = fingerprint_matrices.columns[3:]
    X = fingerprint_matrices.loc[:,features].values
    y = fingerprint_matrices.loc[:,['Molecule']]
    z = fingerprint_matrices.loc[:,['Concentration']]
    a = fingerprint_matrices.loc[:,['Replicate']]
    x = StandardScaler().fit_transform(X)
    pca = PCA(n_components=len(fingerprint_matrices))
    principalComponents = pca.fit_transform(x)
    processed_DF = pd.DataFrame(data = principalComponents, columns = ['principal component' + str(index + 1) for index, item in enumerate(principalComponents)])
    pDFinal = pd.concat([processed_DF, y], axis = 1)
    pDFinal = pd.concat([pDFinal, z], axis = 1)
    pDFinal = pd.concat([pDFinal, a], axis = 1)
    return pDFinal

In [ ]:
experimental_pca = GeneratePCAs(MyTest)
control_pca = GeneratePCAs(control_exp)

In [ ]:
experimental_pca.head()

In [ ]:
def GeneratePCAGraph(molp5_exp, pcas, path_prefix, pc1, pc2):
    fig = plt.figure(figsize = (20,20))
    fig.patch.set_facecolor('white')
    plt.scatter(x = pcas.iloc[:180,pc1], y = pcas.iloc[:180,pc2], label='Replicate 1')
    plt.scatter(x = pcas.iloc[180:360,pc1], y = pcas.iloc[180:360,pc2], label='Replicate 2')
    plt.scatter(x = pcas.iloc[360:540,pc1], y = pcas.iloc[360:540,pc2], label='Replicate 3')
    for i, text in enumerate(pcas['Molecule']):
        plt.annotate(text, (pcas.iloc[:,pc1].iat[i]+0.5, pcas.iloc[:,pc2].iat[i]+0.3), fontsize=16)
    plt.legend(markerscale=2, fontsize=16)
    path = 'PCAs/' + path_prefix + '/' + str(pc1) + 'vs' + str(pc2) + '.png'
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, dpi=300)
    # plt.show()
    plt.close('all')

In [ ]:
GeneratePCAGraph(MyTest, experimental_pca, 'Rep', 1, 2)

In [ ]:
def GenerateAllPCAGraphs(molp5_exp, pca, path_prefix, limit):
    for i in range(1, limit+1):
        GeneratePCAGraph(molp5_exp, pca, path_prefix, i, i+1)

In [ ]:
GenerateAllPCAGraphs(MyTest, experimental_pca, 'Rep', 10)
GenerateAllPCAGraphs(control_exp, control_pca, 'Ctrl', 10)

In [ ]:
# PCA, Separately

In [ ]:
test = MyTest.getFingerprintMatrix(1)
test2 = MyTest.getFingerprintMatrix(2)
test3 = MyTest.getFingerprintMatrix(3)
test_meta = pd.append([])

In [ ]:
features = test.columns[2:]
features_2 = test2.columns[2:]
features_3 = test3.columns[2:]

In [ ]:
X = test.loc[: , features].values
X_2 = test2.loc[: , features_2].values
X_3 = test3.loc[: , features_3].values

In [ ]:
y = test.loc[:,['Molecule']].values
y_2 = test2.loc[:,['Molecule']].values
y_3 = test3.loc[:,['Molecule']].values

In [ ]:
x = StandardScaler().fit_transform(X)
x_2 = StandardScaler().fit_transform(X_2)
x_3 = StandardScaler().fit_transform(X_3)

In [ ]:
pca = PCA(n_components=180)
pca_2 = PCA(n_components=180)
pca_3 = PCA(n_components=180)

In [ ]:
principalComponents = pca.fit_transform(x)
principalComponents_2 = pca.fit_transform(x_2)
principalComponents_3 = pca.fit_transform(x_3)

In [ ]:
pDF = pd.DataFrame(
    data = principalComponents, columns = ['principal component' + str(index + 1) for index, item in enumerate(principalComponents)]
)
pDF_2 = pd.DataFrame(
    data = principalComponents_2, columns = ['principal component' + str(index + 1) for index, item in enumerate(principalComponents_2)]
)
pDF_3 = pd.DataFrame(
    data = principalComponents_3, columns = ['principal component' + str(index + 1) for index, item in enumerate(principalComponents_3)]
)

In [ ]:
PCAFinals = [pDF,pDF_2,pDF_3]

In [ ]:
for entry in PCAFinals:
    fig = plt.figure(figsize = (40,40))
    plt.scatter(x = entry.iloc[:,0],y = entry.iloc[:,1])
    for i, text in enumerate(entry['Molecule']):
        plt.annotate(text, (entry.iloc[:,0].iat[i], entry.iloc[:,1].iat[i]), fontsize=16)
    fig.patch.set_facecolor('white')
    plt.savefig("pctest1v2.png")
    plt.show()

In [ ]:
# Histograms

In [ ]:
# All Reps, Exp.
test_hist_data = MyTest.generateVectorDistances()
fig, ax = plt.subplots(figsize=(10,10))
fig.patch.set_facecolor('white')
test_hist_data.hist(column="Magnitude", by="Concentration", ax=ax, bins=10)
for ax in fig.get_axes():
    ax.set_ylabel("Number of Molecules")
    ax.set_xlabel("Magnitude of Feature Vector")
fig.suptitle("Feature Vector Magnitudes at Different Concentrations", y = 0.97, fontsize = 22)
path = "Histograms/AllReplicates.png"
os.makedirs(os.path.dirname(path), exist_ok=True)
plt.savefig(path)

In [ ]:
# All Reps, Ctrl.
test_hist_data_ctrl = control_exp.generateVectorDistances()
fig, ax = plt.subplots(figsize=(20,20))
fig.patch.set_facecolor('white')
test_hist_data_ctrl.hist(column="Magnitude", by="Concentration", ax=ax, bins=10)
for ax in fig.get_axes():
    ax.set_ylabel("Number of Molecules")
    ax.set_xlabel("Magnitude of Feature Vector")
fig.suptitle("Feature Vector Magnitudes at Different Concentrations", y = 0.955, fontsize = 22)
path = "Histograms/AllControls.png"
os.makedirs(os.path.dirname(path), exist_ok=True)
plt.savefig(path)

In [ ]:
# Individual Reps, Exp.
for rep in range(1, MyTest.replicates+1):
    test_hist_data = MyTest.generateVectorDistances(replicate=rep)
    fig, ax = plt.subplots(figsize=(20,20))
    fig.patch.set_facecolor('white')
    test_hist_data.hist(column="Magnitude", by="Concentration", ax=ax, bins=10)
    for ax in fig.get_axes():
        ax.set_ylabel("Number of Molecules")
        ax.set_xlabel("Magnitude of Feature Vector")
    fig.suptitle("Feature Vector Magnitudes at Different Concentrations", y = 0.955, fontsize = 22)
    path = "Histograms/Rep/Rep" + str(rep) + ".png"
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path)

In [ ]:
# Individual Reps, Ctrl.
for rep in range(1, control_exp.replicates+1):
    test_hist_data = control_exp.generateVectorDistances(replicate=rep)
    fig, ax = plt.subplots(figsize=(20,20))
    fig.patch.set_facecolor('white')
    test_hist_data.hist(column="Magnitude", by="Concentration", ax=ax, bins=10)
    for ax in fig.get_axes():
        ax.set_ylabel("Number of Molecules")
        ax.set_xlabel("Magnitude of Feature Vector")
    fig.suptitle("Feature Vector Magnitudes at Different Concentrations", y = 0.955, fontsize = 22)
    path = "Histograms/Ctrl/Rep" + str(rep) + ".png"
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path)

In [ ]:
# Clustering

In [ ]:
test10 = MyTest.getDistanceMatrix(3, method='euclidean')

In [ ]:
testcluster = AgglomerativeClustering(distance_threshold=0,n_clusters=None).fit(dropColumns(test11, 'Replicate', 'Molecule', 'Concentration', axis=1))

In [ ]:
molecules = [molecule for sublist in [[molecule] * 6 for molecule in MyTest.molecules] for molecule in sublist]

In [ ]:
from scipy.cluster.hierarchy import dendrogram
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    fig = plt.figure(figsize = (20,20))
    fig.patch.set_facecolor('white')
    dendrogram(linkage_matrix, **kwargs)
    plt.xlabel("Well")

In [ ]:
test12 = dropColumns(test11, 'Replicate', 'Molecule', axis=1) 

In [ ]:
fig = plt.figure(figsize=(40,40))
fig.patch.set_facecolor('white')
sns.set(font_scale=0.25)
sns.clustermap(dropColumns(test11, 'Replicate', 'Molecule', axis=1), yticklabels=test11['Molecule'])
plt.savefig("heatmaptest.png",dpi=300)
plt.show()

In [ ]:
plot_dendrogram(testcluster, labels=molecules)
plt.savefig("testden.png", dpi=300)